In [3]:
import pandas as pd
wine_dataset=pd.read_csv("wine-dataset.csv")
wine_dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


# Basic version of Tree

In [5]:
from __future__ import division
from collections import Counter

import math,operator,random,time,copy
import csv


class csvdata():                                                   #Created a class CSV data to deal with the objects of data 
    def __init__(self, classifier):
        self.rows = []
        self.attributes = []
        self.attribute_types = []
        self.classifier = classifier
        self.class_col_index = None

                                                                    #the tree node class which will build the tree
class treeNode():                                                                           
    def __init__(self, is_leaf, classification, attribute_split_index, attribute_split_value, parent, left_child, right_child, height):

   
        self.parent = parent                                                                              
        self.left_child = None
        self.right_child = None
        self.height = None
        self.is_leaf = True

        self.classification = None
        self.attribute_split = None
        self.attribute_split_index = None
        self.attribute_split_value = None



def pre_processing(dataset):                                          #Creating a function to pre process data
    for row in dataset.rows:
        for i in range(len(dataset.rows[0])):
            if dataset.attributes[i] == 'True':
                row[i] = float(row[i])


def compute_decision_tree(dataset, parent_node, classifier):                           #Compute a decision tree function
    node = treeNode(True, None, None, None, parent_node, None, None, 0)
    if (parent_node == None):
        node.height = 0
    else:
        node.height = node.parent.height + 1

   
    ones = count_ones(dataset.rows, dataset.attributes, classifier)          #count the number of rows with classification "1"

    if (len(dataset.rows) == ones):
        node.classification = 1
        node.is_leaf = True
        return node
    elif (ones == 0):
        node.is_leaf = True
        node.classification = 0
        return node
    else:
        node.is_leaf = False

   
    splitting_attribute = None                                                #The index of the attribute we will split on

    
    maximum_info_gain = 0                                                    #The information gain given by the best attribute

    split_val = None
    minimum_info_gain = 0.01

    entropy = calculate_entropy(dataset, classifier)                                  #Calculating Entropy 

    
    for attr_index in range(len(dataset.rows[0])):                                    #for each column of data

        if (dataset.attributes[attr_index] != classifier):
            local_max_gain = 0
            local_split_val = None
            attr_value_list = [example[attr_index] for example in dataset.rows]
            #these are the values we can split on, now we must find the best one
            attr_value_list = list(set(attr_value_list))              #remove duplicates from list of all attribute values

            if(len(attr_value_list) > 100):
                attr_value_list = sorted(attr_value_list)
                total = len(attr_value_list)
                ten_percentile = int(total/10)
                new_list = []
                for x in range(1, 10):
                    new_list.append(attr_value_list[x*ten_percentile])
                attr_value_list = new_list

            for val in attr_value_list:
                                                                        #calculate the gain if we split on this value
                                                      #if gain is greater than local_max_gain, save this gain and this value
                current_gain = calculate_information_gain(attr_index, dataset,val,entropy)

                if (current_gain > local_max_gain):
                    local_max_gain = current_gain
                    local_split_val = val

            if (local_max_gain > maximum_info_gain):
                maximum_info_gain = local_max_gain
                split_val = local_split_val
                splitting_attribute = attr_index

    if (maximum_info_gain <= minimum_info_gain or node.height > 20):
        node.is_leaf = True
        node.classification = classify_leaf(dataset, classifier)
        return node

    node.attribute_split_index = splitting_attribute
    node.attribute_split = dataset.attributes[splitting_attribute]
    node.attribute_split_value = split_val

    left_dataset = csvdata(classifier)
    right_dataset = csvdata(classifier)

    left_dataset.attributes = dataset.attributes
    right_dataset.attributes = dataset.attributes

    left_dataset.attribute_types = dataset.attribute_types
    right_dataset.attribute_types = dataset.attribute_types

    for row in dataset.rows:
        if (splitting_attribute is not None and row[splitting_attribute] >= split_val):
            left_dataset.rows.append(row)
        elif (splitting_attribute is not None):
            right_dataset.rows.append(row)

    node.left_child = compute_decision_tree(left_dataset, node, classifier)
    node.right_child = compute_decision_tree(right_dataset, node, classifier)

    return node


def classify_leaf(dataset, classifier):                                                  #Classify dataset
    ones = count_ones(dataset.rows, dataset.attributes, classifier)
    total = len(dataset.rows)
    zeroes = total - ones
    if (ones >= zeroes):
        return 1
    else:
        return 0



def classify(example, node, class_col_index):                                           #Final evaluation of the data
    if (node.is_leaf == True):
        return node.classification
    else:
        if (example[node.attribute_split_index] >= node.attribute_split_value):
            return classify(example, node.left_child, class_col_index)
        else:
            return classify(example, node.right_child, class_col_index)



def calculate_entropy(dataset, classifier):                                     #Calculate the entropy of the current dataset

    
    ones = count_ones(dataset.rows, dataset.attributes, classifier)          #get count of all the rows with classification 1

    
    total_rows = len(dataset.rows);                                             #get the count of all the rows in the dataset.
    

    
    entropy = 0                                                   #Entropy formula is sum of p*log2(p) P is the probability

    
    p = ones / total_rows                                                  #probability p of classification 1 in total data
    if (p != 0):
        entropy += p * math.log(p, 2)
    
    p = (total_rows - ones)/total_rows                                       #probability p of classification 0 in total data
    if (p != 0):
        entropy += p * math.log(p, 2)

                                                                                     #from the formula
    entropy = -entropy
    return entropy



def calculate_information_gain(attr_index, dataset, val, entropy):        #Calculate the gain of a particular attribute split

    classifier = dataset.attributes[attr_index]
    attr_entropy = 0
    total_rows = len(dataset.rows);
    gain_upper_dataset = csvdata(classifier)
    gain_lower_dataset = csvdata(classifier)
    gain_upper_dataset.attributes = dataset.attributes
    gain_lower_dataset.attributes = dataset.attributes
    gain_upper_dataset.attribute_types = dataset.attribute_types
    gain_lower_dataset.attribute_types = dataset.attribute_types

    for example in dataset.rows:
        if (example[attr_index] >= val):
            gain_upper_dataset.rows.append(example)
        elif (example[attr_index] < val):
            gain_lower_dataset.rows.append(example)

    if (len(gain_lower_dataset.rows) == 0 or len(gain_upper_dataset.rows) == 0):
        return -1

    attr_entropy += len(gain_upper_dataset.rows) * calculate_entropy(gain_upper_dataset, classifier) / total_rows
    attr_entropy += len(gain_lower_dataset.rows) * calculate_entropy(gain_lower_dataset, classifier) / total_rows

    return entropy - attr_entropy




def count_ones(instances, attributes, classifier):                           #count number of rows with classification "1"
    count = 0
    class_col_index = None

    
    for a in range(len(attributes)):                                                #find the index of classifier
        if attributes[a] == classifier:
            class_col_index = a
        else:
            class_col_index = len(attributes) - 1
    for i in instances:
        if i[class_col_index] == "1":
            count += 1
    return count


def validate_tree(node, dataset):
    total = len(dataset.rows)
    correct = 0
    for row in dataset.rows:
        correct += validate_row(node, row)                                                 # validate example
    return correct/total

                                                                                          # Validate row 
def validate_row(node, row):
    if (node.is_leaf == True):
        projected = node.classification
        actual = int(row[-1])
        if (projected == actual):
            return 1
        else:
            return 0
    value = row[node.attribute_split_index]
    if (value >= node.attribute_split_value):
        return validate_row(node.left_child, row)
    else:
        return validate_row(node.right_child, row)

def run():

    f = open("wine-dataset.csv")
    original_file = f.read()
    rowsplit_data = original_file.splitlines()


    data = csvdata("")
    train_set = csvdata("")
    test_set = csvdata("")
    data.rows = [rows.split(',') for rows in rowsplit_data]

    data.attributes = data.rows.pop(0)
    print("Data attributes after removing target label is",data.attributes)

                                                        # true indicates numeric data. false in nominal data
    data.attribute_types = ['true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'false']

    classifier = data.attributes[-1]
    data.classifier = classifier


                                                                                                   # find index of classifier
    for a in range(len(data.attributes)):
        if data.attributes[a] == data.classifier:
            data.class_col_index = a
        else:
            data.class_col_index = range(len(data.attributes))[-1]

                                                                                                   # preprocessing the dataset
    pre_processing(data)

    train_set = copy.deepcopy(data)
    test_set = copy.deepcopy(data)
    validate_set = copy.deepcopy(data)
    
    train_set.rows = []
    test_set.rows = []
    validate_set.rows = []

    K=10
    accuracy = []


    for k in range(1):
        train_set.rows = [x for i, x in enumerate(data.rows) if i % K != k]
        test_set.rows = [x for i, x in enumerate(data.rows) if i % K == k]

        print ("Number of training records: %d" % len(train_set.rows))
        print ("Number of test records: %d" % len(test_set.rows))
        root = compute_decision_tree(train_set, None, classifier)

       
        results = []                                             #Classify the test set using the tree we just constructed
        for instance in test_set.rows:
            result = classify(instance, root, test_set.class_col_index)
            results.append(str(result) == str(instance[-1]))

        
        acc = float(results.count(True))/float(len(results))                                           #Accuracy
        print ("accuracy: %.4f" % acc)

        accuracy.append(acc)
        del root

    accuracy = math.fsum(accuracy)
                                                                              #Writing results to a file
    f = open("result.txt", "w")
    f.write("Accuracy observed : %.4f" % accuracy)
    f.close()

if __name__ == "__main__":
    run()

Data attributes after removing target label is ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
Number of training records: 4408
Number of test records: 490
accuracy: 0.8490


# Implementation using k fold k=10



In [17]:
from __future__ import division
from collections import Counter

import math,operator,random,time,copy
import csv


class csvdata():                                #Created a class CSV data to deal with the objects of data
    def __init__(self, classifier):
        self.rows = []
        self.attributes = []
        self.attribute_types = []
        self.classifier = classifier
        self.class_col_index = None

class treeNode():                                #the tree node class which will build the tree
    def __init__(self, is_leaf, classification, attribute_split_index, attribute_split_value, parent, left_child, right_child, height):

        # tree
        self.parent = parent
        self.left_child = None
        self.right_child = None
        self.height = None
        self.is_leaf = True

        self.classification = None
        self.attribute_split = None
        self.attribute_split_index = None
        self.attribute_split_value = None



def pre_processing(dataset):                                   #Creating a function to pre process data
    for row in dataset.rows:
        for i in range(len(dataset.rows[0])):
            if dataset.attributes[i] == 'True':
                row[i] = float(row[i])


def compute_decision_tree(dataset, parent_node, classifier):          #Compute a decision tree function
    node = treeNode(True, None, None, None, parent_node, None, None, 0)
    if (parent_node == None):
        node.height = 0
    else:
        node.height = node.parent.height + 1

    
    ones = count_ones(dataset.rows, dataset.attributes, classifier)  #count the number of rows with classification "1"

    if (len(dataset.rows) == ones):
        node.classification = 1
        node.is_leaf = True
        return node
    elif (ones == 0):
        node.is_leaf = True
        node.classification = 0
        return node
    else:
        node.is_leaf = False

   
    splitting_attribute = None                                         # The index of the attribute we will split on

   
    maximum_info_gain = 0                                         # The information gain given by the best attribute

    split_val = None
    minimum_info_gain = 0.01

    entropy = calculate_entropy(dataset, classifier)

  
    for attr_index in range(len(dataset.rows[0])):                 #for each column of data

        if (dataset.attributes[attr_index] != classifier):
            local_max_gain = 0
            local_split_val = None
            attr_value_list = [example[attr_index] for example in dataset.rows] # these are the values we can split on, now we must find the best one
            attr_value_list = list(set(attr_value_list)) # remove duplicates from list of all attribute values

            if(len(attr_value_list) > 100):
                attr_value_list = sorted(attr_value_list)
                total = len(attr_value_list)
                ten_percentile = int(total/10)
                new_list = []
                for x in range(1, 10):
                    new_list.append(attr_value_list[x*ten_percentile])
                attr_value_list = new_list

            for val in attr_value_list:
                                                                             # calculate the gain if we split on this value
                                                        # if gain is greater than local_max_gain, save this gain and this value
                current_gain = calculate_information_gain(attr_index, dataset,val,entropy)

                if (current_gain > local_max_gain):
                    local_max_gain = current_gain
                    local_split_val = val

            if (local_max_gain > maximum_info_gain):
                maximum_info_gain = local_max_gain
                split_val = local_split_val
                splitting_attribute = attr_index

    if (maximum_info_gain <= minimum_info_gain or node.height > 20):
        node.is_leaf = True
        node.classification = classify_leaf(dataset, classifier)
        return node

    node.attribute_split_index = splitting_attribute
    node.attribute_split = dataset.attributes[splitting_attribute]
    node.attribute_split_value = split_val

    left_dataset = csvdata(classifier)
    right_dataset = csvdata(classifier)

    left_dataset.attributes = dataset.attributes
    right_dataset.attributes = dataset.attributes

    left_dataset.attribute_types = dataset.attribute_types
    right_dataset.attribute_types = dataset.attribute_types

    for row in dataset.rows:
        if (splitting_attribute is not None and row[splitting_attribute] >= split_val):
            left_dataset.rows.append(row)
        elif (splitting_attribute is not None):
            right_dataset.rows.append(row)

    node.left_child = compute_decision_tree(left_dataset, node, classifier)
    node.right_child = compute_decision_tree(right_dataset, node, classifier)

    return node

                            
def classify_leaf(dataset, classifier):                                                   # Classify dataset
    ones = count_ones(dataset.rows, dataset.attributes, classifier)
    total = len(dataset.rows)
    zeroes = total - ones
    if (ones >= zeroes):
        return 1
    else:
        return 0



def classify(example, node, class_col_index):                                          # Final evaluation of the data
    if (node.is_leaf == True):
        return node.classification
    else:
        if (example[node.attribute_split_index] >= node.attribute_split_value):
            return classify(example, node.left_child, class_col_index)
        else:
            return classify(example, node.right_child, class_col_index)




def calculate_entropy(dataset, classifier):                             # Calculate the entropy of the current dataset

    
    ones = count_ones(dataset.rows, dataset.attributes, classifier)  #get count of all the rows with classification 1

 
    total_rows = len(dataset.rows);                                     #get the count of all the rows in the dataset.
  

    
    entropy = 0                                 #Entropy formula is sum of p*log2(p). Referred the slides. P is the probability

    
    p = ones / total_rows                        #probability p of classification 1 in total data
    if (p != 0):
        entropy += p * math.log(p, 2)
    
    p = (total_rows - ones)/total_rows
    if (p != 0):
        entropy += p * math.log(p, 2)

   
    entropy = -entropy                                #from the formula
    return entropy


                                                               # Calculate the gain of a particular attribute split
def calculate_information_gain(attr_index, dataset, val, entropy):

    classifier = dataset.attributes[attr_index]
    attr_entropy = 0
    total_rows = len(dataset.rows);
    gain_upper_dataset = csvdata(classifier)
    gain_lower_dataset = csvdata(classifier)
    gain_upper_dataset.attributes = dataset.attributes
    gain_lower_dataset.attributes = dataset.attributes
    gain_upper_dataset.attribute_types = dataset.attribute_types
    gain_lower_dataset.attribute_types = dataset.attribute_types

    for example in dataset.rows:
        if (example[attr_index] >= val):
            gain_upper_dataset.rows.append(example)
        elif (example[attr_index] < val):
            gain_lower_dataset.rows.append(example)

    if (len(gain_lower_dataset.rows) == 0 or len(gain_upper_dataset.rows) == 0):
        return -1

    attr_entropy += len(gain_upper_dataset.rows) * calculate_entropy(gain_upper_dataset, classifier) / total_rows
    attr_entropy += len(gain_lower_dataset.rows) * calculate_entropy(gain_lower_dataset, classifier) / total_rows

    return entropy - attr_entropy

           

def count_ones(instances, attributes, classifier):      # count number of rows with classification "1"
    count = 0
    class_col_index = None

    
    for a in range(len(attributes)):                    #find the index of classifier
        if attributes[a] == classifier:
            class_col_index = a
        else:
            class_col_index = len(attributes) - 1
    for i in instances:
        if i[class_col_index] == "1":
            count += 1
    return count


def validate_tree(node, dataset):
    total = len(dataset.rows)
    correct = 0
    for row in dataset.rows:
       
        correct += validate_row(node, row)                # validate example
    return correct/total


def validate_row(node, row):                             
    if (node.is_leaf == True):
        projected = node.classification
        actual = int(row[-1])
        if (projected == actual):
            return 1
        else:
            return 0
    value = row[node.attribute_split_index]
    if (value >= node.attribute_split_value):
        return validate_row(node.left_child, row)
    else:
        return validate_row(node.right_child, row)


def run():

    f = open("wine-dataset.csv")
    original_file = f.read()
    rowsplit_data = original_file.splitlines()


    data = csvdata("")
    train_set = csvdata("")
    test_set = csvdata("")
    data.rows = [rows.split(',') for rows in rowsplit_data]

    data.attributes = data.rows.pop(0)
    print(data.attributes)

                                                               
                                                                    # true indicates numeric data. false in nominal data
    data.attribute_types = ['true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'false']

    classifier = data.attributes[-1]
    data.classifier = classifier



    for a in range(len(data.attributes)):                                # find index of classifier
        if data.attributes[a] == data.classifier:
            data.class_col_index = a
        else:
            data.class_col_index = range(len(data.attributes))[-1]

    print ("classifier is %d" % data.class_col_index)
 
    pre_processing(data)                                                  # preprocessing the dataset

    train_set = copy.deepcopy(data)
    test_set = copy.deepcopy(data)
    validate_set = copy.deepcopy(data)
    
    train_set.rows = []
    test_set.rows = []
    validate_set.rows = []

    K=10                                                                #K fold is done for 10 folds
                                                                        # Stores accuracy of the 10 runs
    accuracy = []

    d=0
    for k in range(K):
        print ("Doing fold ", k)
        train_set.rows = [x for i, x in enumerate(data.rows) if i % K != k]
        test_set.rows = [x for i, x in enumerate(data.rows) if i % K == k]

        print ("Number of training records: %d" % len(train_set.rows))
        print ("Number of test records: %d" % len(test_set.rows))
        root = compute_decision_tree(train_set, None, classifier)

                                                           # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set.rows:
            result = classify(instance, root, test_set.class_col_index)
            results.append(str(result) == str(instance[-1]))

                                                             # Accuracy
        acc = float(results.count(True))/float(len(results))
        print ("accuracy: %.4f" % acc)
        d+=acc

    
    print( "Accuracy  %f " % (d/10))                      #K-fold Accuracy
    
                                    

if __name__ == "__main__":
    run()

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
classifier is 11
Doing fold  0
Number of training records: 4408
Number of test records: 490
accuracy: 0.8490
Doing fold  1
Number of training records: 4408
Number of test records: 490
accuracy: 0.8327
Doing fold  2
Number of training records: 4408
Number of test records: 490
accuracy: 0.8204
Doing fold  3
Number of training records: 4408
Number of test records: 490
accuracy: 0.8265
Doing fold  4
Number of training records: 4408
Number of test records: 490
accuracy: 0.8714
Doing fold  5
Number of training records: 4408
Number of test records: 490
accuracy: 0.8367
Doing fold  6
Number of training records: 4408
Number of test records: 490
accuracy: 0.8122
Doing fold  7
Number of training records: 4408
Number of test records: 490
accuracy: 0.8327
Doing fold  8
Number of training records: 4409
Number of test re

# Report
Initially own version of Decision tree is implemented
Then k fold cross validation was implemented using k=10
k fold is done to select the final model
Here k fold is implemented on one decision tree model
Practically many other models are developed on the same problem statement and k fold accuracy is found for each model
Then the model with best accuracy is implemented on practical scale.